# Análise Exploratória de Dados

In [ ]:
# Importando as bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
print("Bibliotecas importadas com sucesso!")

In [ ]:
# Carregando o DataFrame
df = pd.read_csv("./datasets/police.csv")
print("DataFrame carregado com sucesso!")

In [ ]:
# Visualizando as 5 primeiras linhas
df.head()

In [ ]:
# Quantidade de linhas e colunas
df.shape

In [ ]:
# Verificando o tipo de dado de cada coluna
df.dtypes

In [ ]:
# Consultando linhas com valores faltantes
df.isnull().sum()

In [ ]:
# Removendo a coluna que contém apenas valores ausentes
df.drop("county_name",axis = 1,inplace = True)

In [ ]:
df.columns

In [ ]:
# Método alternativo
df.dropna(axis = "columns", how = "all")

## Homens ou mulheres aceleram com mais frequência?

In [ ]:
# Verificando o total de infrações por excesso de velocidade do sexo masculino e feminino
df[df.violation == "Speeding"].driver_gender.value_counts(normalize=True)

In [ ]:
# Verificando as infrações do sexo masculino em todas as categorias disponíveis
df[df.driver_gender == "M"].violation.value_counts(normalize = True)

In [ ]:
# Verificando as infrações do sexo feminino em todas as categorias disponíveis
df[df.driver_gender == "F"].violation.value_counts(normalize = True)

In [ ]:
# Podemo verificar a contagem de infrações dos dois gêneros ao mesmo tempo usando o groupby
df.groupby("driver_gender").violation.value_counts(normalize = True)

## O gênero afeta quem é revistado durante uma parada?

In [ ]:
# Verificando quando as pessoas foram revistados ou não
df.search_conducted.value_counts(normalize = True)

In [ ]:
# Verificando a média de pessoas revistadas
df.search_conducted.mean()

In [ ]:
# Verificando a média de pessoas revistadas por gênero
df.groupby("driver_gender").search_conducted.mean()

In [ ]:
# Verificando a média de pessoas revistadas por gênero e categoria
df.groupby(["violation","driver_gender"]).search_conducted.mean()

## Por que search_type tem tantos valores nulos?

In [ ]:
# Verificando os valores nulos 
df.isnull().sum()

In [ ]:
# Contando valores nulos ou não nulos
df.search_conducted.value_counts()

In [ ]:
df.search_type.value_counts(dropna = False)

## Durante uma busca, com que frequência o motorista é revistado?

In [ ]:
df["frisk"] = df.search_type.str.contains("Protective Frisk")

In [ ]:
df.frisk.value_counts(dropna = False)

In [ ]:
df.frisk.sum()

In [ ]:
df.frisk.mean()

## Qual ano teve o menor número de paradas?

In [ ]:
# Calculando o número de paradas por ano
df.stop_date.str.slice(0, 4).value_counts()

In [ ]:
combined = df.stop_date.str.cat(df.stop_time, sep = " ")

In [ ]:
df["stop_datetime"] = pd.to_datetime(combined)

In [ ]:
df.stop_datetime.dt.year.value_counts()

## Como a atividade da droga muda de acordo com a hora do dia?

In [ ]:
df.drugs_related_stop.dtype

In [ ]:
df.groupby(df.stop_datetime.dt.hour).drugs_related_stop.mean().plot();

In [ ]:
# Alternativa: contar paradas relacionadas ao uso de drogas por hora
df.groupby(df.stop_datetime.dt.hour).drugs_related_stop.sum().plot();

## A maioria das paradas ocorre à noite?

In [ ]:
df.stop_datetime.dt.hour.value_counts().plot();

In [ ]:
#
df.stop_datetime.dt.hour.value_counts().sort_index().plot();

## Encontrando os dados incorretos na coluna stop_duration e corrigindo

In [ ]:
df.stop_duration.value_counts()

In [ ]:
df.loc[(df.stop_duration == '1') | (df.stop_duration == '2'), 'stop_duration'] = 'NaN'

In [ ]:
df.stop_duration.value_counts(dropna=False)

In [ ]:
df.loc[df.stop_duration == 'NaN', 'stop_duration'] = np.nan

In [ ]:
df.stop_duration.value_counts(dropna=False)

## Qual é a média de cada parada por violação?

In [ ]:
mapping = {'0-15 Min':8, '16-30 Min':23, '30+ Min':45}
df['stop_minutes'] = df.stop_duration.map(mapping)

In [ ]:
# corresponde a value_counts para stop_duration
df.stop_minutes.value_counts()

In [ ]:
df.groupby('violation_raw').stop_minutes.mean()

In [ ]:
df.groupby('violation_raw').stop_minutes.agg(['mean', 'count'])

## Plote os resultados do primeiro GROUPBY do execício anterior

In [ ]:
df.groupby('violation_raw').stop_minutes.mean().plot();

In [ ]:
df.groupby('violation_raw').stop_minutes.mean().plot(kind='bar');

In [ ]:
df.groupby('violation_raw').stop_minutes.mean().sort_values().plot(kind='barh');

## Compare as distribuições de idade para cada violação

In [ ]:
df.groupby('violation').driver_age.describe()

In [ ]:
df.driver_age.plot(kind='hist');

In [ ]:
df.driver_age.value_counts().sort_index().plot();

In [ ]:
df.hist('driver_age', by='violation')
plt.show()

In [ ]:
df.hist('driver_age', by='violation', sharex=True)
plt.show()

In [ ]:
# Essa é melhor que a anterior
df.hist('driver_age', by='violation', sharex=True, sharey=True)
plt.show()

## Finja que você não tem a coluna driver_age e crie-a a partir de driver_age_raw (e chame-a de new_age)

In [ ]:
df['new_age'] = df.stop_datetime.dt.year - df.driver_age_raw
df[['driver_age', 'new_age']].hist()
plt.show()

In [ ]:
df[['driver_age', 'new_age']].describe()

In [ ]:
# Verificando a quantidade de linhas e colunas quando temos idade entre < 15 e 99 >
df[(df.new_age < 15) | (df.new_age > 99)].shape

In [ ]:
df.driver_age_raw.isnull().sum()

In [ ]:
5621-5327

In [ ]:
# driver_age_raw NÃO FALTA, driver_age FALTA
df[(df.driver_age_raw.notnull()) & (df.driver_age.isnull())].head()

In [ ]:
# definir as idades fora desse intervalo como ausentes
df.loc[(df.new_age < 15) | (df.new_age > 99), 'new_age'] = np.nan
df.new_age.equals(df.driver_age)